# Persistent Storage for FPGA tools

This notebook shows how to create, re-create, renew and use a slice with a VM connected to FABNetv4Ext network on EDC on which you can store FPGA tool packages - the transfer of Xilinx tools into VMs can take a substantial amount of time, so it is recommended that projects using FPGAs put their tools into a peristent volume either on a selected site or on centrally-located EDC. The tools are licensed, so it is not advisable to put them in open access. This notebook shows how to deploy an instance of NGINX webserver that adds a password protection to static content retrieved from the node. 

Once this slice is created, it can persist or even go away, however since the volume is persistent, any packages or files you put on it will be preserved and you can always use this notebook to re-attach a new VM to the storage and continue storing those file. You can transfer the packages into this slice using SSH tunnels, and then use a simple curl tool from any VM in FABRIC attached to fabnet to get the packages again and again over high-bandwidth internal dataplane.

Depending on what you want to do with the slice (create/recreate/renew), you may execute all or a subset of the steps below. You __always__ want to execute steps 0 and 1 for the rest of the cells to work.

## Step 0: Import the FABlib Library

In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                         
fablib.show_config();

Log Level,INFO
Log File,/tmp/fablib/fablib.log
Data directory,/tmp/fablib
SSH Command Line,ssh -i {{ _self_.private_ssh_key_file }} -F /Users/baldin/work/fabric_config/ssh_config {{ _self_.username }}@{{ _self_.management_ip }}
Orchestrator,orchestrator.fabric-testbed.net
Credential Manager,cm.fabric-testbed.net
Core API,uis.fabric-testbed.net
Token File,/Users/baldin/workspaces/fabric_token.json
Project ID,bbe0d94c-736b-477a-a2e6-fef9fe7ac9ca
Bastion Host,bastion.fabric-testbed.net
Bastion Username,baldin_0000147304


## Step 1: Initialize slice parameters

You always want to execute this cell for any of the cells below to work. Some cells below can be skipped depending on what you are trying to do.

In [2]:
# Replace with your project's volume name and site
site = "EDC"
storage_name = "ejfat-data"
node_name = "Storage"
slice_name = "Xilinx Tools Storage Slice"
mount_point = "ejfat-data"

# this is the username, password to be used when downloading packages from this node. Change the password!
nginx_user = "fpga_tools"
nginx_password = "changeme!"

##  Step 2: Create the Storage Slice

Create the new slice with a single VM attached to storage and with FABNetv4 auto-configured.

In [3]:
# Create a slice
slice = fablib.new_slice(name=slice_name)

# Add a node with storage and FABNetv4
node = slice.add_node(name=node_name, site=site)
node.add_storage(name=storage_name)
node.add_fabnet()

# Submit the slice
slice.submit();


Retry: 13, Time: 322 sec


ID,0a644f13-f79f-460c-a031-59904989066a
Name,Xilinx Tools Storage Slice
Lease Expiration (UTC),2024-10-08 22:33:14 +0000
Lease Start (UTC),2024-10-07 22:33:14 +0000
Project ID,bbe0d94c-736b-477a-a2e6-fef9fe7ac9ca
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
e3a061fe-42e9-494f-a22b-a1a801f10359,Storage,2,8,10,default_rocky_8,qcow2,edc-w1.fabric-testbed.net,EDC,rocky,2620:0:c80:1003:f816:3eff:fe2f:c779,Active,,ssh -i /Users/baldin/.ssh/slice_key -F /Users/baldin/work/fabric_config/ssh_config rocky@2620:0:c80:1003:f816:3eff:fe2f:c779,/Users/baldin/.ssh/slice_key.pub,/Users/baldin/.ssh/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
dd87e139-d761-4108-9f14-a0fb6c39d6db,FABNET_IPv4_EDC,L3,FABNetv4,EDC,10.132.133.0/24,10.132.133.1,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
Storage-FABNET_IPv4_EDC_nic-p1,p1,Storage,FABNET_IPv4_EDC,100,auto,,1A:00:5C:DB:B1:5A,eth1,eth1,10.132.133.2,6,HundredGigE0/0/0/15



Time to print interfaces 322 seconds


## Step 3: Inspect the slice and get IP address information

If slice already exists, you can just execute this cell assuming Steps 0 and 1 have been executed.

In [4]:
slice = fablib.get_slice(slice_name)

node = slice.get_node(name=node_name)              

node_addr = node.get_interface(network_name=f'FABNET_IPv4_{node.get_site()}').get_ip_addr()

slice.show()
slice.list_nodes()
slice.list_networks()
print(f'Node FABNetV4Ext IP Address is {node_addr}')

ID,0a644f13-f79f-460c-a031-59904989066a
Name,Xilinx Tools Storage Slice
Lease Expiration (UTC),2024-10-08 22:33:14 +0000
Lease Start (UTC),2024-10-07 22:33:14 +0000
Project ID,bbe0d94c-736b-477a-a2e6-fef9fe7ac9ca
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
e3a061fe-42e9-494f-a22b-a1a801f10359,Storage,2,8,10,default_rocky_8,qcow2,edc-w1.fabric-testbed.net,EDC,rocky,2620:0:c80:1003:f816:3eff:fe2f:c779,Active,,ssh -i /Users/baldin/.ssh/slice_key -F /Users/baldin/work/fabric_config/ssh_config rocky@2620:0:c80:1003:f816:3eff:fe2f:c779,/Users/baldin/.ssh/slice_key.pub,/Users/baldin/.ssh/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
dd87e139-d761-4108-9f14-a0fb6c39d6db,FABNET_IPv4_EDC,L3,FABNetv4,EDC,10.132.133.0/24,10.132.133.1,Active,


Node FABNetV4Ext IP Address is 10.132.133.2


## Step 4: Format the Volume (only run this the first time you attach the volume, skip otherwise)

The first time you use your persistent volume it will be a raw block device that needs to be formated.

In [5]:
node = slice.get_node(node_name)

storage = node.get_storage(storage_name)

print(f"Storage Device Name: {storage.get_device_name()}")  

stdout,stderr = node.execute(f"sudo mkfs.ext4 {storage.get_device_name()}")

Storage Device Name: /dev/vdb
Discarding device blocks:          0/131072000 mke2fs 1.45.6 (20-Mar-2020)
                     done                            
Creating filesystem with 1310720000 4k blocks and 163840000 inodes
Filesystem UUID: 4e342289-53f0-4850-8cc3-4806cf090ee1
Superblock backups stored on blocks: 
	32768, 98304, 163840, 229376, 294912, 819200, 884736, 1605632, 2654208, 
	4096000, 7962624, 11239424, 20480000, 23887872, 71663616, 78675968, 
	102400000, 214990848, 512000000, 550731776, 644972544

Allocating group tables:     0/4000          done                            
Writing inode tables:     0/4000          done                            
Creating journal (262144 blocks): done
Writing superblocks and filesystem accounting information:     0/4000          done



## Step 5: Mount the Volume

After you format you volume, you can mount it.  The data in the volume is persisent.  On subseqent uses of the volume, you can mount it and access perviously stored data without formating.

In [6]:
node = slice.get_node(node_name)
storage = node.get_storage(storage_name)

stdout,stderr = node.execute(f"sudo mkdir /mnt/{mount_point}; "
                     f"sudo mount {storage.get_device_name()} /mnt/{mount_point}; "
                     f"df -h")

Filesystem      Size  Used Avail Use% Mounted on
devtmpfs        3.8G     0  3.8G   0% /dev
tmpfs           3.8G     0  3.8G   0% /dev/shm
tmpfs           3.8G  8.7M  3.8G   1% /run
tmpfs           3.8G     0  3.8G   0% /sys/fs/cgroup
/dev/vda5       9.0G  1.3G  7.7G  14% /
/dev/vda2       994M  242M  753M  25% /boot
/dev/vda1        99M  5.8M   94M   6% /boot/efi
tmpfs           770M     0  770M   0% /run/user/1000
/dev/vdb        4.9T   28K  4.6T   1% /mnt/ejfat-data


## Step 6: Install and configure NGINX

Now we install and configure NGINX with a password-protected static content directory. You can transfer the files into this VM via e.g. the bastion host or in some other way, place them into the directory and then from any host within FABRIC with a fabnet network service you can reach this host to retrieve the files. 

In [7]:
command = "sudo dnf install -y nginx httpd-tools"

print('Installing NGINX and apache tools')
stdout, stderr = node.execute(command)

command = "sudo systemctl enable nginx"
print('Enabling NGINX on reboot')
stdout, stderr = node.execute(command)

Installing NGINX and apache tools
Last metadata expiration check: 0:08:12 ago on Tue 08 Oct 2024 02:16:15 AM UTC.
Dependencies resolved.
 Package                       Arch    Version                                   Repo        Size
Installing:
 httpd-tools                   x86_64  2.4.37-65.module+el8.10.0+1842+4a9649e8.2 appstream  111 k
 nginx                         x86_64  1:1.14.1-9.module+el8.4.0+542+81547229    appstream  566 k
Installing dependencies:
 apr                           x86_64  1.6.3-12.el8                              appstream  128 k
 apr-util                      x86_64  1.6.1-9.el8                               appstream  105 k
 gd                            x86_64  2.2.5-7.el8                               appstream  143 k
 jbigkit-libs                  x86_64  2.1-14.el8                                appstream   54 k
 libXpm                        x86_64  3.5.12-11.el8                             appstream   58 k
 libjpeg-turbo                 x86_64  1.5

Create a user with a password to protect the download directory

In [8]:
command = f"sudo htpasswd -bc2 /etc/nginx/htpasswd {nginx_user} {nginx_password}"

print('Setting username and password for downloads')
stdout, stderr = node.execute(command)

Setting username and password for downloads
 Adding password for user fpga_tools
 

Configure NGINX to use SSL with a self-signed cert for static downloads from the mount point where the storage is mounted to.

In [11]:
# install SSL server configuration
node.upload_file('config/ssl-server.conf', 'ssl-server.conf')
command = "sudo mv ssl-server.conf /etc/nginx/conf.d/; sudo chown nginx:nginx /etc/nginx/conf.d/ssl-server.conf"
stdout, stderr = node.execute(command)

# generate a self-signed key/cert
command = 'sudo openssl req -x509 -nodes -days 3650 -newkey rsa:2048 -keyout /etc/nginx/server.key -out /etc/nginx/server.crt ' \
          '-subj="/C=US/ST=NC/L=Chapel Hill/O=UNC/OU=FABRIC/CN=fabric-testbed.net"; ' \
          'sudo chown nginx:nginx /etc/nginx/server.crt /etc/nginx/server.key'
stdout, stderr = node.execute(command)

# install location configuration
nginx_config = """
location /fpga-tools {
      alias /mnt/""" + mount_point + """/static;
      auth_basic "Restricted Access!";     
      auth_basic_user_file htpasswd;
      autoindex on;
      autoindex_format json;
      
      
      dav_methods PUT DELETE MKCOL COPY MOVE;
      dav_access user:rw group:rw all:rw;
      client_max_body_size 0;
      create_full_put_path on;
      client_body_temp_path /tmp/nginx-uploads;
}
"""
# transfer the config to the node
command = f"echo '{nginx_config}' | sudo tee /etc/nginx/default.d/static.conf"
stdout, stderr = node.execute(command)

# create a /mnt/fpga-tools/static/ directory if it doesn't exist already for staging files
command =f"sudo mkdir -p /mnt/{mount_point}/static; sudo chown rocky:rocky /mnt/{mount_point}/static; chmod go+w /mnt/{mount_point}/static"
stdout, stderr = node.execute(command)

# create a /mnt/fpga-tools/static/smartnic-docker-images direcory for ESnet workflow files
command =f"sudo mkdir -p /mnt/{mount_point}/static/smartnic-docker-images; sudo chown rocky:rocky /mnt/{mount_point}/static/smartnic-docker-images; chmod go+w /mnt/{mount_point}/static/smartnic-docker-images"
stdout, stderr = node.execute(command)

# create top-level directory for user artifacts
command =f"sudo mkdir -p /mnt/{mount_point}/static/artifacts; sudo chown rocky:rocky /mnt/{mount_point}/static/artifacts; chmod go+w /mnt/{mount_point}/static/artifacts"
stdout, stderr = node.execute(command)

 Generating a RSA private key
.+++++
...........................................+++++
writing new private key to '/etc/nginx/server.key'
-----
 
location /fpga-tools {
      alias /mnt/ejfat-data/static;
      auth_basic "Restricted Access!";     
      auth_basic_user_file htpasswd;
      autoindex on;
      autoindex_format json;
      
      
      dav_methods PUT DELETE MKCOL COPY MOVE;
      dav_access user:rw group:rw all:rw;
      client_max_body_size 0;
      create_full_put_path on;
      client_body_temp_path /tmp/nginx-uploads;
}



By default SELinux does not want to let NGINX read files from the attached persistent volume. So we need to make it a bit more permissive. The references to the process are here:
- https://www.nginx.com/blog/using-nginx-plus-with-selinux/
- https://relativkreativ.at/articles/how-to-compile-a-selinux-policy-package 

In [12]:
# transfer SELinux policy module file to the node, compile and install it
# the file was originally created using `grep nginx /var/log/audit/audit.log | audit2allow -m nginx > nginx.te` 
# This policy allows nginx to read files and directories in general locations, including the attached storage
# Note that if you have issues with nginx not being able to read files, SELinux is likely to blame
# change `error_log /var/log/nginx/error.log;` to `error_log /var/log/nginx/error.log debug;` in /etc/nginx/nginx.conf
# and then restart NGINX to see what the problem may be
nginx_te = """

module nginx 1.0;

require {
        type init_t;
        type httpd_t;
        type httpd_tmp_t;
        type unlabeled_t;
        class file { create getattr open read rename unlink write };
        class dir { add_name remove_name rmdir write };
}

#============= httpd_t ==============
allow httpd_t unlabeled_t:dir { add_name remove_name write };
allow httpd_t unlabeled_t:file getattr;
allow httpd_t unlabeled_t:file { create open read rename unlink write };

#============= init_t ==============
allow init_t httpd_tmp_t:dir rmdir;

"""

command = f"echo '{nginx_te}' | sudo tee /etc/nginx/nginx.te"
stdout, stderr = node.execute(command)

# compile and install
command = """
sudo checkmodule -M -m -o /etc/nginx/nginx.mod /etc/nginx/nginx.te;
sudo semodule_package -o /etc/nginx/nginx.pp -m /etc/nginx/nginx.mod;
sudo semodule -i /etc/nginx/nginx.pp;
sudo semodule -l | grep nginx
"""
stdout, stderr = node.execute(command)

# add ability to access user home directories
command = "sudo setsebool -P httpd_read_user_content 1"
stdout, stderr = node.execute(command)



module nginx 1.0;

require {
        type init_t;
        type httpd_t;
        type httpd_tmp_t;
        type unlabeled_t;
        class file { create getattr open read rename unlink write };
        class dir { add_name remove_name rmdir write };
}

#============= httpd_t ==============
allow httpd_t unlabeled_t:dir { add_name remove_name write };
allow httpd_t unlabeled_t:file getattr;
allow httpd_t unlabeled_t:file { create open read rename unlink write };

#============= init_t ==============
allow init_t httpd_tmp_t:dir rmdir;


nginx


(RE)Start NGINX

In [13]:
command = "sudo systemctl restart nginx; sudo systemctl status nginx"

stdout, stderr = node.execute(command)

● nginx.service - The nginx HTTP and reverse proxy server
   Loaded: loaded (/usr/lib/systemd/system/nginx.service; enabled; vendor preset: disabled)
   Active: active (running) since Tue 2024-10-08 02:27:30 UTC; 23ms ago
  Process: 13611 ExecStart=/usr/sbin/nginx (code=exited, status=0/SUCCESS)
  Process: 13609 ExecStartPre=/usr/sbin/nginx -t (code=exited, status=0/SUCCESS)
  Process: 13608 ExecStartPre=/usr/bin/rm -f /run/nginx.pid (code=exited, status=0/SUCCESS)
 Main PID: 13613 (nginx)
    Tasks: 3 (limit: 48750)
   Memory: 7.8M
   CGroup: /system.slice/nginx.service
           ├─13613 nginx: master process /usr/sbin/nginx
           ├─13614 nginx: worker process
           └─13615 nginx: worker process

Oct 08 02:27:30 Storage systemd[1]: Starting The nginx HTTP and reverse proxy server...
Oct 08 02:27:30 Storage nginx[13609]: nginx: the configuration file /etc/nginx/nginx.conf syntax is ok
Oct 08 02:27:30 Storage nginx[13609]: nginx: configuration file /etc/nginx/nginx.conf test 

## Step 7: Use the storage

You can move the data into the storage in a number of ways, for example as described in [this page](https://learn.fabric-testbed.net/knowledge-base/transferring-data-to-and-from-your-vms/). Download the tools from Xilinx or other locations and then SCP them into the node using its management IP:

`mylaptop$ scp -F ~/path/to/fabric_ssh_config -i ~/path/to/sliver/private/key Xilinx_Tools/* rocky@[2620:0:c80:1234:f816:3eff:fe7b:2ca1]:/mnt/fpga-tools/static/`

Then from any VM in FABRIC with fabnet network service you can simply curl (or wget) the contents (and it will be very fast) into a new VM:

`newvm$ curl -k -u fpga_tools:secret-password https://<FABNetv4 IP address of the storage VM>/fpga-tools/Vivado/Xilinx_Vivado_Lab_Lin.tar.gz > Xilinx_Vivado_Lab_Lin.tar.gz` 

or

`newvm$ wget --no-check-certificate --user=fpga_tools --password=secret-password https://<FABNetv4 IP address of the storage VM>/fpga-tools/Xilinx_Vivado_Lab_Lin.tar.gz`

You can also list directories - they will be returned in JSON format (for scripting convenience; `fpga-tools/` URL path maps to `/mnt/fpga-tools/static/` file path):

`curl -k -u fpga_tools:secret-password https://<FABNetv4 IP address of the storage VM>/fpga-tools/`

You can write back into the storage also using curl:

`curl -k -u fpga_tools:secret-password -T filename-to-transfer.tar https://fpga-tools-host/fpga-tools/artifacts/`

## Step 8: Extend the slice

If you need to extends the storage slice, you can just execute the following two cells. They display the slice expiration date and optionally extend by 2 weeeks. 

In [ ]:
slice = fablib.get_slice(name=slice_name)
slice.show()

Renew the slice by 14 days

In [10]:
from datetime import datetime
from datetime import timezone
from datetime import timedelta

# Set end host to now plus 14 days
end_date = (datetime.now(timezone.utc) + timedelta(days=14)).strftime("%Y-%m-%d %H:%M:%S %z")

try:
    slice = fablib.get_slice(name=slice_name)

    slice.renew(end_date)
except Exception as e:
    print(f"Exception: {e}")


Retry: 0, Time: 28 sec


ID,0a644f13-f79f-460c-a031-59904989066a
Name,Xilinx Tools Storage Slice
Lease Expiration (UTC),2024-10-21 02:25:10 +0000
Lease Start (UTC),2024-10-07 22:33:14 +0000
Project ID,bbe0d94c-736b-477a-a2e6-fef9fe7ac9ca
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
e3a061fe-42e9-494f-a22b-a1a801f10359,Storage,2,8,10,default_rocky_8,qcow2,edc-w1.fabric-testbed.net,EDC,rocky,2620:0:c80:1003:f816:3eff:fe2f:c779,Active,,ssh -i /Users/baldin/.ssh/slice_key -F /Users/baldin/work/fabric_config/ssh_config rocky@2620:0:c80:1003:f816:3eff:fe2f:c779,/Users/baldin/.ssh/slice_key.pub,/Users/baldin/.ssh/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
dd87e139-d761-4108-9f14-a0fb6c39d6db,FABNET_IPv4_EDC,L3,FABNetv4,EDC,10.132.133.0/24,10.132.133.1,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
Storage-FABNET_IPv4_EDC_nic-p1,p1,Storage,FABNET_IPv4_EDC,100,auto,,1A:00:5C:DB:B1:5A,eth1,eth1,10.132.133.2,6,HundredGigE0/0/0/15



Time to print interfaces 29 seconds


## Step 9: Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
slice = fablib.get_slice(name=slice_name)
slice.delete()